In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
from moveit_msgs.srv import GetStateValidity, GetStateValidityRequest
from moveit_msgs.srv import GetJointWithConstraints, GetJointWithConstraintsRequest
from moveit_msgs.msg import RobotState
from moveit_msgs.msg import Constraints, OrientationConstraint
from moveit_msgs.msg import MoveItErrorCodes
from geometry_msgs.msg import Quaternion, Pose, PoseStamped

In [2]:
moveit_commander.roscpp_initialize(sys.argv)

In [3]:
rospy.init_node('constrained_motion_planning_test', anonymous=True)

In [4]:
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
move_group = moveit_commander.MoveGroupCommander("arm")
joint_names = move_group.get_active_joints()
state_validity_service = rospy.ServiceProxy('/check_state_validity', GetStateValidity)
sample_joint_with_constraints_service = rospy.ServiceProxy('/sample_joint_with_constraints', GetJointWithConstraints)

In [5]:
# load the obstacle
shelf_pose = PoseStamped()
shelf_pose.header.frame_id = "base_link"
shelf_pose.pose.position.x = 0.88
shelf_pose.pose.position.y = 0.23
shelf_pose.pose.position.z = 0
shelf_pose.pose.orientation.x = 0
shelf_pose.pose.orientation.y = 0
shelf_pose.pose.orientation.z = 1
shelf_pose.pose.orientation.w = 0
scene.add_mesh('shelf', shelf_pose, 'constraint_scene/Gruppe_21/model.stl', (0.02, 0.03, 0.02))


In [6]:
horizontal_constraint = Constraints()
horizontal_constraint.name = "use_equality_constraints"

oc = OrientationConstraint()

oc.parameterization = OrientationConstraint.ROTATION_VECTOR;
oc.header.frame_id = "base_link";
oc.header.stamp = rospy.Time(0)
oc.link_name = "wrist_roll_link";
constrained_quaternion = Quaternion();
constrained_quaternion.x = 0.0
constrained_quaternion.y = 0.0
constrained_quaternion.z = 0.0
constrained_quaternion.w = 1.0
oc.orientation = constrained_quaternion
oc.weight = 1.0

oc.absolute_x_axis_tolerance = 0.1
oc.absolute_y_axis_tolerance = 0.1
oc.absolute_z_axis_tolerance = 2 * 3.1415
horizontal_constraint.orientation_constraints.append(oc)

# need to set in-hand pose
in_hand_pose = Pose()
in_hand_pose.position.x = 0.0
in_hand_pose.position.y = 0.0
in_hand_pose.position.z = 0.0
in_hand_pose.orientation.x = 0.0
in_hand_pose.orientation.y = 0.0
in_hand_pose.orientation.z = 0.0
in_hand_pose.orientation.w = 1.0
horizontal_constraint.in_hand_pose = in_hand_pose

In [7]:
sample_request = GetJointWithConstraintsRequest()
sample_request.constraints = horizontal_constraint
sample_request.group_name = "arm"
init_sample = sample_joint_with_constraints_service(sample_request)
goal_sample = sample_joint_with_constraints_service(sample_request)
print("error code of init sample ", init_sample.error_code)
print("error code of target sample ", goal_sample.error_code)

('error code of init sample ', val: 1)
('error code of target sample ', val: 1)


In [8]:
def getProperJointState(all_name, all_joint, selected_name):
    result = []
    for sn in selected_name:
        result.append(all_joint[all_name.index(sn)])
    return result

In [9]:
init_joint = getProperJointState(init_sample.solution.joint_state.name, init_sample.solution.joint_state.position, joint_names)
goal_joint = getProperJointState(goal_sample.solution.joint_state.name, goal_sample.solution.joint_state.position, joint_names)

In [10]:
print init_joint
print goal_joint
goal_joint = [-0.08726646259971647, 0.6806784082777885, 1.2566370614359172, -2.1467549799530254, -2.426007660272118, -1.9024088846738192, 2.181661564992912]

[1.56, 0.23763285161161862, -0.08012094135867276, -0.8483469202420837, -1.6825601657057776, 1.687096173120022, 0.8855510842115883]
[-1.605, -1.040907759982167, -1.1034493622637114, -1.7059415772530977, 0.9177229605422603, -0.6106022411103424, 1.7762815776320824]


In [11]:
moveit_robot_state = RobotState()
moveit_robot_state.joint_state.name = joint_names
moveit_robot_state.joint_state.position = init_joint

move_group.set_planner_id('CBIRRTConfigDefault')
move_group.set_planning_time(10.0)
move_group.set_path_constraints(horizontal_constraint)
move_group.set_in_hand_pose(in_hand_pose)
move_group.set_clean_planning_context_flag(True)

move_group.set_start_state(moveit_robot_state)
move_group.set_joint_value_target(goal_joint)
result = move_group.plan()

In [ ]:
import math
degrees = [-5,39,72,-123,-139,-109,125]
radians = [math.radians(d) for d in degrees]

print(radians)